In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

%matplotlib inline
sns.set(rc={'figure.figsize': [20, 20]}, font_scale=1.4)

In [ ]:
df = pd.read_csv('../input/thyroid-disease-data-set/hypothyroid.csv')
df

In [4]:
df.head

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df["binaryClass"].value_counts()

In [ ]:
df["binaryClass"]=df["binaryClass"].map({"P":0,"N":1})

In [ ]:
df["pregnant"].value_counts()

In [ ]:
df["pregnant"].value_counts()

In [ ]:
df

In [ ]:
#df['target'].isnull().sum()

In [ ]:
df['sex'].isnull().sum()

In [ ]:
df["TBG"].value_counts()

In [ ]:
del df["TBG"]

In [ ]:
df=df.replace({"?":np.NAN})

In [ ]:
df.isnull().sum()

In [ ]:
df["sex"].value_counts()

In [ ]:
df=df.replace({"F":1,"M":0})

In [ ]:
df["referral source"].value_counts()

In [ ]:
del df["referral source"]

In [ ]:
df.info()

In [ ]:
df["T3 measured"].value_counts()

In [ ]:
df["TT4 measured"].value_counts()

In [ ]:
df["FTI measured"].value_counts()


In [ ]:
df["TBG measured"].value_counts()

In [ ]:
df["binaryClass"].value_counts()

In [ ]:
df.dtypes


In [ ]:
cols = df.columns[df.dtypes.eq('object')]
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df.dtypes

In [ ]:
df.isnull().sum()


In [ ]:
df['T4U measured'].mean()

In [ ]:
df['T4U measured'].fillna(df['T4U measured'].mean(), inplace=True)

In [ ]:
df['sex'].fillna(df['sex'].mean(), inplace=True)

In [ ]:
df['age'].fillna(df['age'].mean(), inplace=True)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

In [ ]:
df['TSH'] = imputer.fit_transform(df[['TSH']])

In [ ]:
df['T3'] = imputer.fit_transform(df[['T3']])

In [ ]:
df['TT4'] = imputer.fit_transform(df[['TT4']])

In [ ]:
df['T4U'] = imputer.fit_transform(df[['T4U']])

In [ ]:
df['FTI'] = imputer.fit_transform(df[['FTI']])

In [ ]:
df.isnull().sum()

In [ ]:
df


In [ ]:
df.columns

In [ ]:
import seaborn as sns

%matplotlib inline
sns.set(rc={'figure.figsize': [8, 8]}, font_scale=1.2)

In [ ]:
sns.distplot(df['age'

In [ ]:
sns.distplot(df['sex'])

In [ ]:
sns.distplot(df['T3'])

In [ ]:
sns.distplot(df['TT4'])

In [ ]:
sns.distplot(df['T4U'])

In [ ]:
sns.distplot(df['FTI'])

In [ ]:
sns.distplot(df['TBG measured'])

In [ ]:
sns.jointplot(x='age', y='TT4', data=df, kind='scatter', height=8, color='m')

In [ ]:
sns.jointplot(x='age', y='TT4', data=df, kind='reg', height=8, color='m')

In [ ]:
sns.distplot(df['age'], kde=False, bins=30, color='m')

In [ ]:
sns.countplot(x='binaryClass', data=df, palette='rocket')

In [ ]:
sns.countplot(x='binaryClass', data=df, hue='sex', palette='BuPu')


In [ ]:
sns.stripplot(x="binaryClass", y="age", data=df, palette="viridis")

In [ ]:
sns.boxplot(x='binaryClass', y='age', data=df)

In [ ]:
sns.jointplot(x='FTI', y='binaryClass', data=df, kind='scatter', height=8, color='m')

In [ ]:
df_corr = df.corr()
df_corr

In [ ]:
sns.heatmap(df_corr, cmap='viridis', linecolor='k', linewidths=2, annot=True)

In [ ]:
#df.dropna(axis = 0, inplace=True)

In [ ]:
x = df.drop('binaryClass', axis=1)
y = df['binaryClass']

In [ ]:
x

In [ ]:
y

In [ ]:
x = sm.add_constant(x)
results = sm.OLS(y,x).fit()
results.summary()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
x.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [ ]:
x.shape

In [ ]:
x.shape[1]

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=[x.shape[1]], activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(63, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png') 

In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',
                         patience = 20,
                         verbose = 1,
                         factor = 0.75,
                         min_lr = 1e-10)

mcp = ModelCheckpoint('model.h5')

es = EarlyStopping(verbose=1, patience=20)

In [ ]:
%%time
history = model.fit(x=x_train, y=y_train, epochs=100, callbacks=[lrd, mcp, es], batch_size=64, validation_split=0.1)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_pred = model.predict(x_test)
y_pred

In [ ]:
y_pred[1]


In [ ]:
y_test



In [ ]:
%%time
model.evaluate(x_test, y_test)

In [ ]:
model.predict(x_test)[50]

In [ ]:
model.predict_classes(x_test)[50]

In [ ]:
y_test.iloc[50]

In [ ]:
model.predict(x_test)[70]

In [ ]:
model.predict_classes(x_test)[70]

In [ ]:
y_test.iloc[70]

In [ ]:
model.predict(sc.transform([[42.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.00,132,1.00,1.00,109.0,1.0,0.88,.100,110.00,0.00,0,1]]))

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.legend(['train', 'test'], loc='upper left')
plt.show()

# # summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
model.save('model.h5')

In [ ]:
import joblib

In [ ]:
joblib.dump(sc, 'scaler.pkl')

In [ ]:
sc = joblib.load('scaler.pkl')

In [ ]:
%%time
from tensorflow.keras.models import load_model
model = load_model('model.h5')
model.predict(sc.transform([[42.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.00,132,1.00,1.00,109.0,1.0,0.88,.100,110.00,0.00,0,1]]))